In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set(style="darkgrid")
import pickle

In [2]:
df_customer = pd.read_csv('customer.csv')
df_click = pd.read_csv('click_stream.csv')
df_product = pd.read_csv('product.csv', on_bad_lines='skip')
df_trans = pd.read_csv('transactions.csv')

# Cleaning Data

## Dataset Customer

In [3]:
#mengubah tipe data yang tidak sesuai
df_customer['birthdate'] = pd.to_datetime(df_customer['birthdate'])
df_customer['first_join_date'] = pd.to_datetime(df_customer['first_join_date'])

## Product Dataset

In [4]:
#mengubah nama kolom gender product, agar berbeda dengan gender customer
df_product.gender = df_product.rename(columns = {'gender' : 'gender_product'}, inplace = True)

#Mengubah tipe data kolom year
df_product["year"] = pd.to_numeric(df_product["year"], errors='coerce')

fixed_product = df_product.copy()

# Mengubah null value dengan mode yang sesuai dengan articleType (Face Moisturisers)
fixed_product.at[15526, "baseColour"] = "White"

# Mengubah null value dengan mode yang sesuai dengan articleType (Jeans)
fixed_product.at[6206, "baseColour"] = "Blue"
fixed_product.at[9516, "baseColour"] = "Blue"
fixed_product.at[12763, "baseColour"] = "Blue"
fixed_product.at[15708, "baseColour"] = "Blue"
fixed_product.at[20209, "baseColour"] = "Blue"
fixed_product.at[34553, "baseColour"] = "Blue"

# Mengubah null value dengan mode yang sesuai dengan articleType (Perfume and Body Mist)
fixed_product.at[14291, "baseColour"] = "Blue"
fixed_product.at[17268, "baseColour"] = "Blue"
fixed_product.at[17639, "baseColour"] = "Blue"
fixed_product.at[23484, "baseColour"] = "Blue"
fixed_product.at[37531, "baseColour"] = "Blue"
fixed_product.at[40675, "baseColour"] = "Blue"
fixed_product.at[43260, "baseColour"] = "Blue"
fixed_product.at[44224, "baseColour"] = "Blue"

fixed_product['season'] = fixed_product['season'].fillna(fixed_product['season'].mode()[0])

#Mengubah missing value pada kolom year dengan mode
fixed_product['year'] = fixed_product['year'].fillna(fixed_product['year'].mode()[0])

# Mengubah null value dengan mode yang sesuai dengan articleType (Perfume and Body Mist) dan baseColour
fixed_product.at[3677, "productDisplayName"] = "Lino Perros Women Maroon Clutch"
fixed_product.at[22089, "productDisplayName"] = "Rocia Women Brown Clutch"

#Mengubah missing value pada kolom productDisplayName dengan mode
fixed_product['productDisplayName'] = fixed_product['productDisplayName'].fillna(fixed_product['productDisplayName'].mode()[0])

#Mengubah missing value pada kolom usage dengan mode
fixed_product['usage'] = fixed_product['usage'].fillna(fixed_product['usage'].mode()[0])

## Transactions Dataset

In [5]:
# Ubah Tipe Data shipment_date_limit
df_trans['shipment_date_limit'] = pd.to_datetime(df_trans['shipment_date_limit'])
#convert waktu UTC +0 menjadi waktu local
df_trans['shipment_date_limit'] = df_trans['shipment_date_limit'].dt.tz_convert("Asia/Jakarta")
#extract waktu
df_trans['shipment_date_limit'] = pd.to_datetime(df_trans['shipment_date_limit'], format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')
# Ubah Tipe Data shipment_date_limit
df_trans['shipment_date_limit'] = pd.to_datetime(df_trans['shipment_date_limit'])

# Ubah tipe data created_id
df_trans['created_at'] = pd.to_datetime(df_trans['created_at'])
#convert waktu UTC +0 menjadi waktu local
df_trans['created_at'] = df_trans['created_at'].dt.tz_convert("Asia/Jakarta")
#extract waktu
df_trans['created_at'] = pd.to_datetime(df_trans['created_at'], format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')
df_trans['created_at'] = pd.to_datetime(df_trans['created_at'])

# Handling Missing Value
df_trans['promo_code'].fillna('NOPROMO', inplace=True)

# Hapus spesial character pada kolom produtc_metadata
df_trans['product_metadata'].replace("[^\w\s]", ' ', regex=True, inplace=True)

# Membuat Kolom id dengan product_id isi dari kolom product metadata
df_trans['id'] = df_trans.product_metadata.str[15:21]
df_trans['id'] = pd.to_numeric(df_trans['id'], errors='coerce')

# Membuat Kolom qty dengan quantity dari kolom product metadata
df_trans['qty'] = df_trans.product_metadata.str[34:36]
df_trans['qty'].replace("[^\w\s]", ' ', regex=True, inplace=True)
df_trans['qty'] = pd.to_numeric(df_trans['qty'], errors='coerce')

# Membuat Kolom price dengan item_price dari kolom product metadata
df_trans['price'] = df_trans.product_metadata.str[51:58]
df_trans['qty'].replace("[^\w\s]", ' ', regex=True, inplace=True)
df_trans['price'] = pd.to_numeric(df_trans['price'], errors='coerce')

## Click Stream Dataset

In [6]:
#mengubah tipe data yang tidak sesuai
df_click['event_time'] = pd.to_datetime(df_click['event_time'])
#convert waktu UTC +0 menjadi waktu local
df_click['event_time'] = df_click['event_time'].dt.tz_convert("Asia/Jakarta")
#extract waktu
df_click['event_time'] = pd.to_datetime(df_click['event_time'], format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S')
df_click['event_time'] = pd.to_datetime(df_click['event_time'])

# Merging Dataset

In [7]:
# Merge dataset
df_1 = pd.merge(df_click,df_trans,on='session_id', how='left')
df_1 = pd.merge(df_1, fixed_product,on='id', how='left' )
df_1 = pd.merge(df_1, df_customer, on='customer_id', how='left')

In [8]:
# membuat features tahun dilakukannya pembelian
df_1["purchase_year"] = df_1.created_at.dt.year

gender_codes = {'F': 0, 'M': 1}
df_1['gender'] = df_1.gender.map(gender_codes)
df_1.gender = df_1.gender.astype('category')

df_1['device_type'].fillna('unknown', inplace=True)
device_codes = {'Android': 0, 'iOS': 1, 'unknown': 2}
df_1['device_type'] = df_1.device_type.map(device_codes)
df_1.device_type = df_1.device_type.astype('float')

# Ubah kolom traffic_source menjadi 0 untuk web dan 1 untuk mobile
traf_cols = {'WEB': 0, 'MOBILE': 1}
df_1['traffic_source'] = df_1['traffic_source'].map(traf_cols)
df_1['traffic_source'] = pd.to_numeric(df_1['traffic_source'], errors='coerce')

df_1['age'] =  df_1.created_at.dt.year.max() - df_1.birthdate.dt.year

#membuat feature shipment fee customer
ongkir = df_1.groupby("customer_id")[["shipment_fee"]].sum()
ongkir = ongkir.reset_index()
ongkir.columns = ["customer_id","ongkir"]

# membuat feature jumlah promo yang telah customer gunakan
total_promo = df_1.groupby("customer_id")[["promo_amount"]].sum()
total_promo = total_promo.reset_index()
total_promo.columns = ["customer_id","total_promo"]

recency = df_1.groupby(['customer_id'])['created_at'].max()
recency = pd.DataFrame(data = recency).reset_index()

recency['recent_days'] = recency['created_at'].max() - recency['created_at']
recency['recent_days'] = recency['recent_days'].dt.days

frequency = pd.DataFrame(df_1.groupby('customer_id')['booking_id'].count())

monetary = pd.DataFrame(df_1[['customer_id','total_amount']].groupby('customer_id')['total_amount'].sum())

In [9]:
df_rfm = pd.merge(recency, frequency, on='customer_id')
df_rfm = pd.merge(df_rfm, monetary, on='customer_id')

df_rfm = df_rfm.rename(columns = {'booking_id' : 'frequency'})
df_rfm = df_rfm.rename(columns = {'total_amount' : 'monetary'})

In [10]:
df_rfm['R_rank'] = df_rfm['recent_days'].rank(ascending=False)
df_rfm['F_rank'] = df_rfm['frequency'].rank(ascending=True)
df_rfm['M_rank'] = df_rfm['monetary'].rank(ascending=True)

# normalizing the rank of the customers
df_rfm['R_rank_norm'] = (df_rfm['R_rank']/df_rfm['R_rank'].max())*100
df_rfm['F_rank_norm'] = (df_rfm['F_rank']/df_rfm['F_rank'].max())*100
df_rfm['M_rank_norm'] = (df_rfm['F_rank']/df_rfm['M_rank'].max())*100

df_rfm.drop(columns=['R_rank', 'F_rank', 'M_rank'], inplace=True)

# referensi: https://www.geeksforgeeks.org/rfm-analysis-analysis-using-python/
df_rfm['RFM_Score'] = 0.15*df_rfm['R_rank_norm']+0.28 * df_rfm['F_rank_norm']+0.57*df_rfm['M_rank_norm']
df_rfm['RFM_Score'] *= 0.05
df_rfm = df_rfm.round(2)

df_rfm["Customer_segment"] = np.where(df_rfm['RFM_Score'] >
                                    4.5, "Top Customers",
                                    (np.where(df_rfm['RFM_Score'] > 4,
                                        "High value Customer",
                                        (np.where(df_rfm['RFM_Score'] > 3,
                            "Medium Value Customer",
                            np.where(df_rfm['RFM_Score'] > 1.6,
                            'Low Value Customers', 'Lost Customers'))))))

In [11]:
table = df_1.pivot_table(
    values="session_id", 
    index='customer_id', 
    columns='event_name', 
    aggfunc='count')

act_cust = table.reset_index()

In [12]:
pembelian_awal = df_1.groupby(['customer_id'])['created_at'].min()
pembelian_awal = pd.DataFrame(data = pembelian_awal).reset_index()
pembelian_awal = pembelian_awal.rename(columns = {'created_at' : 'initial_purchase_date'})
pembelian_akhir = df_1.groupby(['customer_id'])['created_at'].max()
pembelian_akhir = pd.DataFrame(data = pembelian_akhir).reset_index()
pembelian_akhir = pembelian_akhir.rename(columns = {'created_at' : 'last_purchase_date'})

In [13]:
new_df = pd.merge(pembelian_awal, pembelian_akhir, on='customer_id')
df_rfm = pd.merge(new_df, df_rfm, on='customer_id')

In [14]:
df_rfm["tenure"] = (df_rfm.last_purchase_date - df_rfm.initial_purchase_date).dt.days/30
df_rfm["monthly_spend"] = np.round(df_rfm.monetary / df_rfm.tenure, 2)
df_rfm["monthly_spend"].replace([np.inf, -np.inf], np.nan, inplace=True)
df_rfm["monthly_spend"].fillna(df_rfm.monetary, inplace=True)

# membuat feature berapa lama antara pembelian pertama dan pembelian selanjutnya
df_rfm["freq_diff"] = df_rfm['recent_days'] / df_rfm['frequency']

# Membuat data dari 3 bulan terakhir
data_last_months = df_1[(df_1['created_at'] > '2022-01-01 00:00:00') & (df_1['created_at'] <= '2022-04-01 00:00:00')]
s = pd.merge(df_rfm, data_last_months, left_on='customer_id',right_on='customer_id',how='left')
df_rfm['purchased_3m'] = s.customer_id.notnull().astype(int)
df_rfm.purchased_3m = df_rfm.purchased_3m.astype('category')

# Jumlah order per customer
order_count = df_1.groupby(['customer_id'])['qty'].count().sort_values(ascending = False).reset_index()
order_count = order_count.rename(columns = {'qty' : 'order_count'})

# Kategori order
kategori_order = []
for i in order_count['order_count']:
    if i <= 5:
        kategori_order.append('0') # Sedikit
    else:
        kategori_order.append('1') # Banyak
kategori_order = pd.DataFrame({'kategori_order' : kategori_order})
order_count = pd.concat([order_count, kategori_order], axis = 1)

In [15]:
# Merge order_count ke df_rfm
df_rfm = pd.merge(df_rfm, order_count, on='customer_id')

In [16]:
# Selish Tanggal pembelian
df_rfm['selisih_tanggal'] = (df_rfm.last_purchase_date - df_rfm.initial_purchase_date).dt.days
# Rata-rata pembelian per customer 1x 
df_rfm['rata_rata_pembelian'] = df_rfm['selisih_tanggal'] / df_rfm['order_count']

# Kategori rata-rata pembelian
kategori_rata_rata_pembelian = []
for i in df_rfm['rata_rata_pembelian']:
    if (i <= 30) & (i >= 1):
        kategori_rata_rata_pembelian.append('1') # Aktif 
    else:
        kategori_rata_rata_pembelian.append('0') # Tidak Aktif
kategori_rata_rata_pembelian = pd.DataFrame({'kategori_rata_rata_pembelian' : kategori_rata_rata_pembelian})
df_rfm = pd.concat([df_rfm, kategori_rata_rata_pembelian], axis = 1)

# Rename kolom kategori_rata_rata_pembelian
df_rfm = df_rfm.rename(columns = {'kategori_rata_rata_pembelian' : 'churn'})

In [17]:
fixed_df = df_1.drop_duplicates(subset=['customer_id'])
df_rfm = pd.merge(df_rfm, ongkir, on='customer_id', how='left')
df_rfm = pd.merge(df_rfm, total_promo, on='customer_id', how='left')

df = pd.merge(fixed_df, df_rfm, on='customer_id', how='inner')
df = pd.merge(df, act_cust, on='customer_id', how='inner')

In [18]:
#mengubah nilai null pada aktivitas customer dengan 0
df = df.replace(np.nan, 0)

## Outlier Detection

In [19]:
cols = ['purchase_year', 'ongkir','total_promo', 'recent_days', 'frequency', 'monetary', 'RFM_Score', 'tenure', 'monthly_spend', 'freq_diff', 'ADD_PROMO', 'ADD_TO_CART']

# Cek Outlier dengan IQR
def outlier_iqr(data):
    outliers = []
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    batas_bawah = q1 - 1.5 * iqr
    batas_atas = q3 + 1.5 * iqr
    return batas_bawah, batas_atas
    for i in data:
        if i < batas_bawah or i > batas_atas:
            outliers.append(i)
    return outliers
data_outlier = {}
for col in cols:
    data_outlier[col] = outlier_iqr(df[col])
    print('Outlier (',col,') : ',len(data_outlier[col]),' outlier',data_outlier[col])

Outlier ( purchase_year ) :  2  outlier (2013.5, 2025.5)
Outlier ( ongkir ) :  2  outlier (-2977500.0, 5402500.0)
Outlier ( total_promo ) :  2  outlier (-758722.875, 1320854.125)
Outlier ( recent_days ) :  2  outlier (-441.0, 839.0)
Outlier ( frequency ) :  2  outlier (-316.5, 583.5)
Outlier ( monetary ) :  2  outlier (-206697464.5, 366524419.5)
Outlier ( RFM_Score ) :  2  outlier (-2.380000000000001, 7.380000000000001)
Outlier ( tenure ) :  2  outlier (-48.21666666666666, 89.25)
Outlier ( monthly_spend ) :  2  outlier (-10478224.811250001, 20558036.578750003)
Outlier ( freq_diff ) :  2  outlier (-23.22815092383107, 39.111008201357464)
Outlier ( ADD_PROMO ) :  2  outlier (-6.5, 13.5)
Outlier ( ADD_TO_CART ) :  2  outlier (-52.0, 100.0)


In [20]:
# Handling (Monetary)
q1 = df['monetary'].quantile(0.25)
q3 = df['monetary'].quantile(0.75)
iqr = q3 - q1
batas_bawah = q1 - 1.5 * iqr
batas_atas = q3 + 1.5 * iqr
print('batas_bawah : ',batas_bawah)
print('batas_atas : ',batas_atas)
df['monetary'] = np.where(df['monetary'] > batas_atas, batas_atas, df['monetary'])
df['monetary'] = np.where(df['monetary'] < batas_bawah, batas_bawah, df['monetary'])

batas_bawah :  -206697464.5
batas_atas :  366524419.5


In [21]:
# Handling (Monthly Spend)
q1 = df['monthly_spend'].quantile(0.25)
q3 = df['monthly_spend'].quantile(0.75)
iqr = q3 - q1
batas_bawah = q1 - 1.5 * iqr
batas_atas = q3 + 1.5 * iqr
print('batas_bawah : ',batas_bawah)
print('batas_atas : ',batas_atas)
df['monthly_spend'] = np.where(df['monthly_spend'] > batas_atas, batas_atas, df['monthly_spend'])
df['monthly_spend'] = np.where(df['monthly_spend'] < batas_bawah, batas_bawah, df['monthly_spend'])

batas_bawah :  -10478224.811250001
batas_atas :  20558036.578750003


In [22]:
# Handling Ongkir
q1 = df['ongkir'].quantile(0.25)
q3 = df['ongkir'].quantile(0.75)
iqr = q3 - q1
batas_bawah = q1 - 1.5 * iqr
batas_atas = q3 + 1.5 * iqr
print('batas_bawah : ',batas_bawah)
print('batas_atas : ',batas_atas)
df['ongkir'] = np.where(df['ongkir'] > batas_atas, batas_atas, df['ongkir'])
df['ongkir'] = np.where(df['ongkir'] < batas_bawah, batas_bawah, df['ongkir'])

batas_bawah :  -2977500.0
batas_atas :  5402500.0


In [23]:
# Handling Total Promo
q1 = df['total_promo'].quantile(0.25)
q3 = df['total_promo'].quantile(0.75)
iqr = q3 - q1
batas_bawah = q1 - 1.5 * iqr
batas_atas = q3 + 1.5 * iqr
print('batas_bawah : ',batas_bawah)
print('batas_atas : ',batas_atas)
df['total_promo'] = np.where(df['total_promo'] > batas_atas, batas_atas, df['total_promo'])
df['total_promo'] = np.where(df['total_promo'] < batas_bawah, batas_bawah, df['total_promo'])

batas_bawah :  -758722.875
batas_atas :  1320854.125


# Data Preprocessing

### Splitting Data

In [24]:
df['churn'] = df['churn'].astype('int')
df['purchased_3m'] = pd.to_numeric(df['purchased_3m'], errors='coerce')
df['gender'] = pd.to_numeric(df['gender'], errors='coerce')
df['device_type'] = pd.to_numeric(df['device_type'], errors='coerce')
df['churn'] = df['churn'].astype('int')

In [25]:
X = (df[['selisih_tanggal','purchase_year','tenure','RFM_Score','total_promo','monetary','ongkir','monthly_spend','freq_diff','recent_days','promo_code','payment_method']])
y = (df['churn'])

In [26]:
# One Hot Encoding
categorical_cols = X.select_dtypes('object').columns.tolist()
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoder.fit(df[categorical_cols])
encoded_cols = list(encoder.get_feature_names(categorical_cols))
X[encoded_cols] = encoder.transform(X[categorical_cols])

In [ ]:
# Save Encoder for Deployment
import pickle
pickle.dump(encoder, open('encoder.pkl', 'wb'))


In [27]:
X.drop(categorical_cols, axis=1, inplace=True)
pd.set_option('display.max_columns', None)
#from imblearn.over_sampling import SMOTE
#from collections import Counter
#X, y = SMOTE().fit_resample(X, y)

In [35]:
# To Csv X dan y (X = Feature, y = Target)
# y.to_csv('y.csv', index=False)
# X.to_csv('Deploy.csv', index=False)

## Load Saved Scaler

In [29]:
# Load Saved Scaler
scaler = pickle.load(open('irlandia-scaler.pkl', 'rb'))
X = scaler.fit_transform(X)

## Load Saved Model

In [37]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [31]:
pkl_filename = "irlandia_grid_Knn.pkl"
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

# Predict data baru
score = pickle_model.score(X_test, y_test)
print("Test score: {0:.2f} %".format(100 * score))
y_predict = pickle_model.predict(X_test)
print("Predicted values:")
print(y_predict)

Test score: 95.08 %
Predicted values:
[1 0 1 ... 1 1 1]


In [32]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.92      0.93      0.93      3865
           1       0.96      0.95      0.95      6276

    accuracy                           0.94     10141
   macro avg       0.94      0.94      0.94     10141
weighted avg       0.94      0.94      0.94     10141

